In [1]:
# Utilities
import fiona
from shapely.geometry import shape, mapping  # Ensure this is imported
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import getpass

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)

from utils import plot_image

C:\Users\Momin.Ashraf\AppData\Local\miniconda3\envs\pycopern\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load json or txt file

In [11]:
with open('credentials.txt', 'r') as f:
    credentials = dict(line.strip().split('=', 1) for line in f if '=' in line)
# Access the credentials
password = credentials.get('COPERNICUS_PASSWORD')
username = credentials.get('COPERNICUS_USERNAME')
aoi_path = credentials.get(r'AOIPATH')
start_date = credentials.get('STARTDATE')
end_date = credentials.get('ENDDATE') 
cloud_filter = credentials.get('CLOUDFILTER')
output_name = credentials.get('OUTPUTNAME')

Convert polygon to wkt

In [25]:
import fiona
from shapely.geometry import shape, mapping, Polygon

# Open the shapefile or GeoPackage
# Open the shapefile or GeoPackage
with fiona.open(aoi_path) as shp:
    for feature in shp:
        geometry = shape(feature["geometry"])

        # Check if the geometry has Z and strip it
        if geometry.has_z:
            # Create a 2D version of the geometry
            geometry_2d = Polygon([(x, y) for x, y, z in geometry.exterior.coords])
            wkt = geometry_2d.wkt

            # Adjust WKT format to match the desired style
            wkt_fixed = wkt.replace('POLYGON (', 'POLYGON(').replace(', ', ',').replace(' ', ' ')
          

aoi_1 = wkt_fixed
aoi = f"{aoi_1}'"
print(aoi)


POLYGON((-1.06482362615344 51.4775993109376,-1.0648176309286 51.4767329603795,-1.06360890264833 51.4766474543255,-1.0635910538134 51.4775435502895,-1.06482362615344 51.4775993109376))'


Send json request to copernicus API

In [26]:
# API request URL
json = requests.get(f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq 'SENTINEL-2' and OData.CSC.Intersects(area=geography'SRID=4326;{aoi}) and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value lt {cloud_filter}) and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq 'S2MSI2A') and ContentDate/Start gt {start_date}T00:00:00.000Z and ContentDate/Start lt {end_date}T00:00:00.000Z&$orderby=ContentDate/Start desc").json()

df = pd.DataFrame.from_dict(json['value'])

# Print only specific columns
columns_to_print = ['Id', 'Name','GeoFootprint',]  
df[columns_to_print]

,Id,Name,GeoFootprint
0,f3028d33-e7a2-4a71-acdb-8c53e4d6169c,S2B_MSIL2A_20250125T111239_N0511_R137_T30UXC_2...,"{'type': 'Polygon', 'coordinates': [[[-1.53211..."
1,aa94321a-711d-4ab7-ba7e-246307d9662a,S2B_MSIL2A_20241126T111319_N0511_R137_T30UXC_2...,"{'type': 'Polygon', 'coordinates': [[[-1.53211..."
2,0b2b31b5-f4ba-4049-8167-7317f9e5ff10,S2A_MSIL2A_20240823T110621_N0511_R137_T30UXC_2...,"{'type': 'Polygon', 'coordinates': [[[-1.53211..."
3,4f9e4f25-0da0-4d87-8f8b-2c86036773ed,S2B_MSIL2A_20240729T110619_N0511_R137_T30UXC_2...,"{'type': 'Polygon', 'coordinates': [[[-1.53211..."
4,63696991-86e4-42d2-ae50-fda3b478abd8,S2B_MSIL2A_20240719T110619_N0510_R137_T30UXC_2...,"{'type': 'Polygon', 'coordinates': [[[-1.53211..."
5,62b8af62-dc4e-40d5-b15e-c6a523928b38,S2B_MSIL2A_20240520T110619_N0510_R137_T30UXC_2...,"{'type': 'Polygon', 'coordinates': [[[-1.53211..."
6,60c790ab-fa08-4768-b05e-43edf1db0bd1,S2A_MSIL2A_20240126T111331_N0510_R137_T30UXC_2...,"{'type': 'Polygon', 'coordinates': [[[-1.53211..."


Select most recent image ID

In [29]:
df2 = df.iloc[0]['Id']
df2

'f3028d33-e7a2-4a71-acdb-8c53e4d6169c'

authenticate account to prepare download

In [18]:
def get_access_token(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Access token creation failed. Response from the server was: {r.json()}"
        )
    return r.json()["access_token"]

# Automatically use credentials from the file
access_token = get_access_token(username, password)

order image and download to working directory

In [117]:
url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({df2})/$value"

headers = {"Authorization": f"Bearer {access_token}"}

session = requests.Session()
session.headers.update(headers)
response = session.get(url, headers=headers, stream=True)

print(f'{output_name} is currently being downloaded')

with open(f"{output_name}.zip", "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            file.write(chunk)

print(f'{output_name} has now been exported as a .zip file in your working directory') 

IMAGEOUTPUT is currently being downloaded
IMAGEOUTPUT has now been exported as a .zip file in your working directory
